<a href="https://colab.research.google.com/github/ArthiyaD/TroubleShooters/blob/Development/ACG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim==3.8.3

In [ ]:
import nltk
nltk.download('stopwords')
import tensorflow as tf
import pandas as pd
import numpy as np
import pickle as pkl
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from gensim.models import Word2Vec,KeyedVectors
from IPython.display import Image, display
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing import image, sequence
from tensorflow.keras.applications.resnet50 import ResNet50

In [ ]:
!gdown 1Y6v9wfUkibbYBTso534eK9uWJ3JzXKO9
!gdown 1Z4C6sVs5EhnwhW6I8m-DKhYJw3e7A6WX
!gdown 1Xv3CVAUuQPZbgQJqWJbuiPx14hcApcj5
!gdown 1YN4m-gece5itGTzIUdZC1t8pn1sWWZOc
!gdown 1jNFcFytMGUnG3oTgIYttilXD5B7ORjlD

In [ ]:
word_vect = '/content/GoogleNews-vectors-negative300.bin.gz'
model_text = '/content/model_text_categorize.h5'
model_image = '/content/saved_model.hp5'

loaded_model = tf.keras.models.load_model(model_text)
saved_model = tf.keras.models.load_model(model_image)
model = KeyedVectors.load_word2vec_format(word_vect, 
                                          binary=True,
                                          limit=1000000)

In [ ]:
resnet = ResNet50(include_top=False, weights='imagenet',
                  input_shape=(224,224,3), pooling='avg')

In [ ]:
with open('/content/word_2_indices.pickle', 'rb') as handle:
     word_2_indices = pkl.load(handle)

with open('/content/indices_2_word.pickle', 'rb') as handle:
    indices_2_word = pkl.load(handle)